**Nom Etudiant : TEMBELY**  

**Prenom Etudiant: Khadidia**  

**Classe ::M1 RSI oneline**  


# Travail à Faire:
Télécharger le Datasets sur le lien Drive : https://drive.google.com/file/d/1M1EwR7InB10Dzw7iU0MG5v4QWVeCaD48/view?usp=sharing   

Repondre les questions ci-dessous avec le maximum de precisions et de détails.   
Remplir `FILL_IN` avec les methodes qui correspondent à la réponse adéquate

### Revenus des achats
1. Extraire les revenus d'achat pour chaque événement
2. Filtrer les événements dont le revenu n'est pas nul
3. Vérifiez quels sont les types d'événements qui génèrent des revenus
4. Supprimez la colonne inutile

### Revenus par Traffic  
Obtenir les 3 sources de trafic générant le revenu total le plus élevé.  
5. Revenus cumulés par source de trafic  
7. Obtenir les 3 principales sources de trafic par revenu total  
6. Nettoyer les colonnes de revenus pour avoir deux décimales  
8. Sauvegarder les données  


##### Methods
- DataFrame: `select`, `drop`, `withColumn`, `filter`, `dropDuplicates`,  `groupBy`, `sort`, `limit`
- Column: `isNotNull`, `alias`, `desc`, `cast`, `operators`

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder.
                        appName("Session de Rattrapage Big Data").
                        config("spark.ui.port", "0").
                        master("yarn").
                        getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.43.129:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1652432035917)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@202d497e


In [2]:
val eventsPath = "datasets/events/events.json"

eventsPath: String = datasets/events/events.json


In [3]:
val eventsDF = spark.read.json(eventsPath)
eventsDF.show()

+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|
+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
|  macOS|{null, null, null}|  warranty|        1593878899217692|1593878946592107|      {Montrose, MI}|                  []|        google|          1593878899217692|UA000000107379500|
|Windows|{null, null, null}|     press|        1593876662175340|1593877011756535|   {Northampton, MA}|                  []|        google|          1593876662175340|UA000000107359357|
|  macOS|{null, null, null}|  add_item|        1593878792892652|1593878815459100

eventsDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 8 more fields]


### 1. Extraire les revenus d'achat pour chaque événement
Ajouter une nouvelle colonne **`revenue`** en faisant l'extration de **`ecommerce.purchase_revenue_in_usd`**

In [13]:
// j'ai utilisé la methode withColumn pour ajouter la nouvelle colonne revenue
val revenueDF = eventsDF.withColumn("revenue",col("ecommerce.purchase_revenue_in_usd"))
revenueDF.show()

+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|  macOS|{null, null, null}|  warranty|        1593878899217692|1593878946592107|      {Montrose, MI}|                  []|        google|          1593878899217692|UA000000107379500|   null|
|Windows|{null, null, null}|     press|        1593876662175340|1593877011756535|   {Northampton, MA}|                  []|        google|          1593876662175340|UA000000107359357|   null|
|  macOS|{null, null, null}|  add_item| 

revenueDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 9 more fields]


### 2. Filtrer les événements dont le revenu n'est pas null

In [14]:
/*j'ai appliqué la methode filter au dataframe revenueDF et isNotNull à la  colonne revenue pour retourner
un dataframe purchasesDF dont les événements ont des revenus non null*/ 
val purchasesDF = revenueDF.filter($"revenue".isNotNull)
purchasesDF.show()

+---------+--------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|   device|     ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+---------+--------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|    Linux|{1195.0, 1, 1}|  finalize|        1593878893766134|1593878897648871|       {Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
|      iOS|{1045.0, 1, 1}|  finalize|        1593878485345763|1593878487460247|       {Detroit, MI}|[{null, M_STAN_Q,...|      facebook|          1593877230282722|UA000000107364432| 1045.0|
|  Android| {595.0, 1, 1}|  finalize|        15938

purchasesDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 9 more fields]


### 3. Vérifiez quels sont les types d'événements qui génèrent des revenus
Trouvez des valeurs **`event_name`** uniques dans **`purchasesDF`**. Il y a deux façons de faire :
- Sélectionnez "event_name" et recupérer les enregistrements distincts
- Supprimez les enregistrements en double en fonction de "event_name" uniquement

In [15]:
/*j'ai appliqué la methode dropDuplicates("event_name") au dataframe  purchasesDF " 
pour supprimer les enregistrements en double  en fonction de "event_name" */ 
val distinctDF = purchasesDF.dropDuplicates("event_name")

distinctDF.show()

+------+--------------+----------+------------------------+----------------+-------------+--------------------+--------------+--------------------------+-----------------+-------+
|device|     ecommerce|event_name|event_previous_timestamp| event_timestamp|          geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+------+--------------+----------+------------------------+----------------+-------------+--------------------+--------------+--------------------------+-----------------+-------+
| Linux|{1195.0, 1, 1}|  finalize|        1593878893766134|1593878897648871|{Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
+------+--------------+----------+------------------------+----------------+-------------+--------------------+--------------+--------------------------+-----------------+-------+



distinctDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 9 more fields]


### 4. Supprimez la colonne inutile
Puisqu'il n'y a qu'un seul type d'événement, supprimez **`event_name`** de **`purchasesDF`**.

In [16]:
//j'ai appliqué la methode drop  au dataframe  purchasesDF  pour supprimer la colonne  "event_name"  
val cleanDF = purchasesDF.drop("event_name")
cleanDF.show

+---------+--------------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|   device|     ecommerce|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+---------+--------------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|    Linux|{1195.0, 1, 1}|        1593878893766134|1593878897648871|       {Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
|      iOS|{1045.0, 1, 1}|        1593878485345763|1593878487460247|       {Detroit, MI}|[{null, M_STAN_Q,...|      facebook|          1593877230282722|UA000000107364432| 1045.0|
|  Android| {595.0, 1, 1}|        1593877930076602|1593878966392505|  {East Chicago, IN}|[{null, M_STAN_T

cleanDF: org.apache.spark.sql.DataFrame = [device: string, ecommerce: struct<purchase_revenue_in_usd: double, total_item_quantity: bigint ... 1 more field> ... 8 more fields]


### 5. Revenus cumulés par source de trafic
- Grouper par **`traffic_source`**
- Obtenir la somme de **`revenue`** comme **`total_rev`**
- Obtenir la moyenne de **`revenue`** comme **`avg_rev`**

N'oubliez pas d'importer toutes les fonctions intégrées nécessaires.

In [18]:
/*J' ai utilisè la fonction d'agrégation agg(), pour calculer 2 agrégations à la fois sur une seule instruction 
en utilisant les fonctions d'agrégation Spark SQL sum() et avg() 
alias  pour nomer la somme de revenue comme total_rev et la moyenne de revenue comme avg_rev*/
val trafficDF = cleanDF.groupBy("traffic_source")
                         .agg(
                            sum("revenue").alias("total_rev"),
                            avg("revenue").alias("avg_rev"))
trafficDF.show()


+--------------+---------+-----------------+
|traffic_source|total_rev|          avg_rev|
+--------------+---------+-----------------+
|     instagram|   8160.0|           1020.0|
|        direct|   9129.0|         1141.125|
|       youtube|   3040.0|           1520.0|
|         email|  36935.4|998.2540540540541|
|      facebook|  12952.0|996.3076923076923|
|        google|  28936.0|964.5333333333333|
+--------------+---------+-----------------+



trafficDF: org.apache.spark.sql.DataFrame = [traffic_source: string, total_rev: double ... 1 more field]


### 6. Obtenir les trois principales sources de trafic par revenu total
- Trier par **`total_rev`** dans l'ordre décroissant
- Se limiter aux trois premières lignes

In [19]:
/*j'ai triè  total_rev dans l'ordre décroissant en utilisant les methodes orderBy et desc et limit(3) 
pour se limiter aux trois premières lignes*/
val topTrafficDF = trafficDF.orderBy(col("total_rev").desc).limit(3)
topTrafficDF.show()

+--------------+---------+-----------------+
|traffic_source|total_rev|          avg_rev|
+--------------+---------+-----------------+
|         email|  36935.4|998.2540540540541|
|        google|  28936.0|964.5333333333333|
|      facebook|  12952.0|996.3076923076923|
+--------------+---------+-----------------+



topTrafficDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [traffic_source: string, total_rev: double ... 1 more field]


### 7. Limitez les colonnes de revenus à deux décimales pointés
- Modifier les colonnes **`avg_rev`** et **`total_rev`** pour les convertir en des nombres avec deux décimales pointés

In [23]:
// TODO
finalDF = topTrafficDF.FILL_IN
finalDF.show()

### 8. Sauvegarder les données 
Sauvegarder les données sous le format parquet

In [ ]:
// TODO
finalDF.write.parquet("ExamenRattrapage.parquet")